# Freespace Segmentation using TAO UNET

MIT License
 SPDX-FileCopyrightText: Copyright (c) 2022 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
 SPDX-License-Identifier: MIT

 Permission is hereby granted, free of charge, to any person obtaining a
 copy of this software and associated documentation files (the "Software"),
 to deal in the Software without restriction, including without limitation
 the rights to use, copy, modify, merge, publish, distribute, sublicense,
 and/or sell copies of the Software, and to permit persons to whom the
 Software is furnished to do so, subject to the following conditions:

 The above copyright notice and this permission notice shall be included in
 all copies or substantial portions of the Software.

 THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
 IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
 FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
 THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
 LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
 FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
 DEALINGS IN THE SOFTWARE.


## Learning Objectives
In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Collect synthetic dataset using Isaac Sim for freespace segmentation
* Use a pretrained "peoplesemseg" model from NVIDIA NGC and train on Isaac Sim generated dataset
* Use NVIDIA TAO toolkit to reduce inference latency using method such as pruning and post training quantization
* Run Inference on the trained and optimized model and visualize the inferences
* Export the trained model to a .etlt file for deployment with Isaac ROS
* Run inference on the exported .etlt model to verify deployment using TensorRT

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

![Freespace Seg workflow](images/intor_to_workflow.png)

### Table of Contents

This notebook shows an example use case of UNet Binary Semantic Segmentation using Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables ](#head-0)
1. [Installing the TAO launcher ](#head-1)
2. [Prepare dataset and pre-trained model ](#head-2)
	1. [Set up Isaac Sim ](#head-2-1)
	2. [Synthetic Data Generation ](#head-2-2)
	3. [ Visualize the Groundtruth Masks ](#head-2-3)
	4. [Post Processing on Synthetic Data ](#head-2-4)
	5. [Download pre-trained model ](#head-2-5)
3. [Provide freespace segmentation training specification ](#head-3)
4. [Run TAO training ](#head-4)
5. [Evaluate trained models ](#head-5)
6. [Visualizing Inferences ](#head-6)
7. [Prune the trained model ](#head-7)
8. [Retrain the pruned model ](#head-8)
9. [Evaluate the retrained model ](#head-9)
10. [Fine-Tune on Real - World Data ](#head-10)
11. [Evaluate the retrained model ](#head-11)
12. [Model Export ](#head-12)
13. [Verify the deployed model ](#head-13)

## 0. Set up env variables <a class="anchor" id="head-0"></a>

When using the purpose-built pretrained models from NGC, please make sure to set the `$KEY` environment variable to the key as mentioned in the model overview. Failing to do so, can lead to errors when trying to load them as pretrained models.

In this experiment; synthitically generated data using NVIDIA Isaac Sim will be stored at `$DATA_DOWNLOAD_DIR` and we will split it in training and validation set. `$USER_EXPERIMENT_DIR` will be used for inputs and outputs models or logs from train, prune, eval experiments. 

*Note: Please make sure to remove any stray artifacts/files from the `$USER_EXPERIMENT_DIR` or `$DATA_DOWNLOAD_DIR` paths, that may have been generated from previous experiments. Having checkpoint files etc may interfere with creating a training graph for a new experiment.*

*Note: This notebook currently is by default set up to run training using 1 GPU. To use more GPU's please update the env variable `$NUM_GPUS` accordingly*


In [ ]:
# Setting up env variables for cleaner command line commands.
import os

%set_env KEY=tlt_encode
%set_env GPU_INDEX=0
%set_env USER_EXPERIMENT_DIR=/workspace/tao-experiments/freespace
%set_env DATA_DOWNLOAD_DIR=/workspace/tao-experiments/data

# Set this path if you don't run the notebook from the samples directory.
%env NOTEBOOK_ROOT=/home/karma/synthetic_data/freespace_main/freespace_seg

# Please define this local project directory that needs to be mapped to the TAO docker session.
# The dataset expected to be present in $LOCAL_PROJECT_DIR/data, while the results for the steps
# in this notebook will be stored at $LOCAL_PROJECT_DIR/unet
# !PLEASE MAKE SURE TO UPDATE THIS PATH!.
%env LOCAL_PROJECT_DIR=/home/karma/tao_experiments

# !PLEASE MAKE SURE TO UPDATE THIS PATH!.
# Point to the 'deps' folder in samples from where you are launching notebook inside unet folder.
%env PROJECT_DIR=/workspace/tao-experiments/robot_freespace_seg_Isaac_TAO/deps

os.environ["LOCAL_DATA_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "data"
)
os.environ["LOCAL_EXPERIMENT_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "freespace"
)

# The sample spec files are present in the same path as the downloaded samples.
os.environ["LOCAL_SPECS_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "specs"
)
%set_env SPECS_DIR=/workspace/tao-experiments/robot_freespace_seg_Isaac_TAO/specs

! ls -l $LOCAL_DATA_DIR

The cell below maps the project directory on your local host to a workspace directory in the TAO docker instance, so that the data and the results are mapped from in and out of the docker. For more information please refer to the [launcher instance](https://docs.nvidia.com/tao/tao-toolkit/tao_launcher.html) in the user guide.

When running this cell on AWS, update the drive_map entry with the dictionary defined below, so that you don't have permission issues when writing data into folders created by the TAO docker.

```json
drive_map = {
    "Mounts": [
            # Mapping the data directory
            {
                "source": os.environ["LOCAL_PROJECT_DIR"],
                "destination": "/workspace/tao-experiments"
            },
            # Mapping the specs directory.
            {
                "source": os.environ["LOCAL_SPECS_DIR"],
                "destination": os.environ["SPECS_DIR"]
            },
        ],
    "DockerOptions": {
        "user": "{}:{}".format(os.getuid(), os.getgid())
    }
}
```

In [ ]:
# Mapping up the local directories to the TAO docker.

import json
mounts_file = os.path.expanduser("~/.tao_mounts.json")

# Define the dictionary with the mapped drives
drive_map = {
    "Mounts": [
        # Mapping the data directory
        {
            "source": os.environ["LOCAL_PROJECT_DIR"],
            "destination": "/workspace/tao-experiments"
        },
        # Mapping the specs directory.
        {
            "source": os.environ["LOCAL_SPECS_DIR"],
            "destination": os.environ["SPECS_DIR"]
        },
    ]
}

# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(drive_map, mfile, indent=4)

In [ ]:
!cat ~/.tao_mounts.json

## 1. Installing the TAO launcher <a class="anchor" id="head-1"></a>
The TAO launcher is a python package distributed as a python wheel listed in the `nvidia-pyindex` python index. You may install the launcher by executing the following cell.

Please note that TAO Toolkit recommends users to run the TAO launcher in a virtual env with python 3.6.9. You may follow the instruction in this [page](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a python virtual env using the `virtualenv` and `virtualenvwrapper` packages. Once you have setup virtualenvwrapper, please set the version of python to be used in the virtual env by using the `VIRTUALENVWRAPPER_PYTHON` variable. You may do so by running

```sh
export VIRTUALENVWRAPPER_PYTHON=/path/to/bin/python3.x
```
where x >= 6 and <= 8

We recommend performing this step first and then launching the notebook from the virtual environment. In addition to installing TAO python package, please make sure of the following software requirements:
* python >=3.6.9 < 3.8.x
* docker-ce > 19.03.5
* docker-API 1.40
* nvidia-container-toolkit > 1.3.0-1
* nvidia-container-runtime > 3.4.0-1
* nvidia-docker2 > 2.5.0-1
* nvidia-driver > 455+

Once you have installed the pre-requisites, please log in to the docker registry nvcr.io by following the command below

```sh
docker login nvcr.io
```

You will be triggered to enter a username and password. The username is `$oauthtoken` and the password is the API key generated from `ngc.nvidia.com`. Please follow the instructions in the [NGC setup guide](https://docs.nvidia.com/ngc/ngc-overview/index.html#generating-api-key) to generate your own API key.

Please note that TAO Toolkit recommends users to run the TAO launcher in a virtual env with python >=3.6.9. You may follow the instruction in this [page](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a python virtual env using the virtualenv and virtualenvwrapper packages.

In [ ]:
!export VIRTUALENVWRAPPER_PYTHON=/usr/bin/python3

In [ ]:
! source ~/Envs/launcher/bin/activate


In [ ]:
# SKIP this step IF you have already installed the TAO launcher.
!pip3 install nvidia-pyindex
!pip3 install nvidia-tao

In [ ]:
# View the versions of the TAO launcher
!tao info

## 2. Prepare dataset and pre-trained model <a class="anchor" id="head-2"></a>

### A. Set up Isaac Sim <a class="anchor" id="head-2-1"></a>
* Follow the installation guide to setup [Isaac Sim](https://docs.omniverse.nvidia.com/app_isaacsim/app_isaacsim/install_basic.html#workstation-setup)
* Run the [Omniverse Launcher](https://docs.omniverse.nvidia.com/app_isaacsim/app_isaacsim/install_basic.html#isaac-sim-setup-native-workstation-launcher) and click Launch button in Isaac Sim App Selector
* In Isaac Sim App Selector, select the 'Open in File Browser' option which will lead you to folder where installation took place. 
* Get the path of this folder, it will be needed for running the python scripts for SDG

### B. Synthetic Data Generation <a class="anchor" id="head-2-2"></a>

#### We need to set the correct paths for the scripts to generate data in the simple room and warehouse scenario:
* Open the script at 'IsaacSim/sdg/simple_room_freespace.sh'
* Set the Isaac Sim Path corresponding to the location of the folder found in Step A
* The script and YAML path correspond to the location where this project is cloned
* The number of images and randomization corresponding to the floor and wall can be set as arguments to the Python Scripts
* The correpsonding YAML file contains the objects to add in the scene, texture locations and groups of floor and wall prims

Repeat the steps for the 'IsaacSim/sdg/warehouse_freespace.sh' script as well.

#### IMPORTANT:
Make sure to create floor and wall randomization folders which contain the appropriate images for their textures. For better results, textures should be similar to the real world environment where the model will be deployed. Upload these folders to your Nucleus Server and specify their path in the YAML file

In [ ]:
## MAKE SURE THE PATHS IN THE SCRIPTS ARE SET CORRECTLY AS DESCRIBED ABOVE
!chmod +x IsaacSim/sdg/simple_room_freespace.sh
!chmod +x IsaacSim/sdg/warehouse_freespace.sh

!./IsaacSim/sdg/simple_room_freespace.sh
!./IsaacSim/sdg/warehouse_freespace.sh

### C.  Visualize the Groundtruth Masks <a class="anchor" id="head-2-3"></a>

In [ ]:
import os 
import matplotlib.image as mpimg
import matplotlib.pyplot as plt


def visualize_images(image_folder):
    plt.figure(figsize=(20,10))
    columns = 5
    img_list = os.listdir(image_folder)
    for num in range(0,5):
        plt.subplot(2, columns, num + 1)
        image_path = img_list[num]
        image = mpimg.imread(os.path.join(image_folder,image_path))
        plt.imshow(image)   

In [ ]:
rgb_folder = 'IsaacSim/simple_room_freespace/Viewport/rgb'
semantic_folder = 'IsaacSim/simple_room_freespace/Viewport/semantic'
visualize_images(rgb_folder)
visualize_images(semantic_folder)

### D. Post Processing on Synthetic Data <a class="anchor" id="head-2-4"></a>
* Now we need to convert the semantic masks in the format which TAO expects, each pixel is represented by the class ID corresponding to it.
* Run the scripts below to post process the data, the viewport directory is where images and labels for each scenario was saved
* Change it to the approprate path if you changed the OUTPUT_FOLDER in the previous scripts whicle generating data

In [ ]:
!python3 IsaacSim/utils/postProcess_syntheticData.py --scenario simple_room \
                                                     --viewport_directory IsaacSim/simple_room_freespace/Viewport

!python3 IsaacSim/utils/postProcess_syntheticData.py --scenario warehouse \
                                                     --viewport_directory IsaacSim/warehouse_freespace/Viewport

In [ ]:
# Path to the warehouse data folder
warehouse_data = 'IsaacSim/warehouse_freespace'

# Path to the simple room data folder
simple_room_data = 'IsaacSim/simple_room_freespace'

# Path to the folder where training images are stored for TAO
train_images = os.path.join(os.environ["LOCAL_PROJECT_DIR"], 'data/train_images')

# Path to the folder where training labels are stored for TAO
train_labels = os.path.join(os.environ["LOCAL_PROJECT_DIR"], 'data/train_labels')

# Create the train images and train labels directories
os.mkdir(train_images)
os.mkdir(train_labels)

# Copy the warehouse and simple room data to the TAO folders
!python3 IsaacSim/utils/copy_data.py --sim_source_folder $warehouse_data --tao_train_images_folder $train_images \
                                    --tao_train_labels_folder $train_labels

!python3 IsaacSim/utils/copy_data.py --sim_source_folder $simple_room_data --tao_train_images_folder $train_images \
                                    --tao_train_labels_folder $train_labels

In [ ]:
# Path to validation images for TAO
validation_images = os.path.join(os.environ["LOCAL_PROJECT_DIR"], 'data/validation_images')

# Path to validation labels for TAO
validation_labels = os.path.join(os.environ["LOCAL_PROJECT_DIR"], 'data/validation_labels')

# Create the valdiation directories for TAO
os.mkdir(validation_images)
os.mkdir(validation_labels)

# Split the data for training and validation
!python3 IsaacSim/utils/split_data.py --train_images $train_images --train_labels $train_labels \
            --val_images $validation_images --val_labels $validation_labels

### E. Download pre-trained model <a class="anchor" id="head-2-5"></a>

We will use NGC CLI to get the pre-trained models. For more details, go to ngc.nvidia.com and click the SETUP on the navigation bar. 

We will be using `peoplesemseg` model based on vanilla unet segementation. We download `trainable` model from NVIDIA NGC. 

In [ ]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/pretrained_peoplesemseg/

In [ ]:
# Pull pretrained model from NGC

!wget --quiet --show-progress --progress=bar:force:noscroll --auth-no-challenge --no-check-certificate \
        https://api.ngc.nvidia.com/v2/models/nvidia/tao/peoplesemsegnet/versions/trainable_v1.0/files/peoplesemsegnet.tlt \
        -P  $LOCAL_EXPERIMENT_DIR/pretrained_peoplesemseg/

In [ ]:
print("Check that model is downloaded into dir.")
!ls -l $LOCAL_EXPERIMENT_DIR/pretrained_peoplesemseg

## 3. Provide freespace segmentation training specification <a class="anchor" id="head-3"></a>

* Images and Masks path
    * In order to use the newly generated images, masks folder update the dataset_config parameter in the spec file at `$SPECS_DIR/spec_vanilla_unet_train.txt` 
    * Update the train, val images and masks paths. The test only requires the images path. 
* Pre-trained models
* Augmentation parameters for on the fly data augmentation
* Other training (hyper-)parameters such as batch size, number of epochs, learning rate etc.


In [ ]:
!cat $LOCAL_SPECS_DIR/spec_vanilla_unet_train.txt

## 4. Run TAO training <a class="anchor" id="head-4"></a>
* Provide the sample spec file and the output directory location for models
* WARNING: training will take several hours or one day to complete

In [ ]:
print("For multi-GPU, change --gpus based on your machine.")
!tao unet train --gpus=1 --gpu_index=$GPU_INDEX \
              -e $SPECS_DIR/spec_vanilla_unet_train.txt \
              -r $USER_EXPERIMENT_DIR/semseg_experiment_unpruned \
              -m  $USER_EXPERIMENT_DIR/pretrained_peoplesemseg/peoplesemsegnet.tlt \
              -n freespace_unpruned \
              -k tlt_encode 

Unet supports restarting from checkpoint. In case, the training job is killed prematurely, you may resume training from the closest checkpoint by simply re-running the same command line. Please do make sure to use the same number of GPUs when restarting the training.

In [ ]:
print('Model for every epoch at checkpoint_interval mentioned in the spec file:')
print('---------------------')
!ls -ltrh $LOCAL_EXPERIMENT_DIR/semseg_experiment_unpruned/
!ls -ltrh $LOCAL_EXPERIMENT_DIR/semseg_experiment_unpruned/weights

## 5. Evaluate trained models <a class="anchor" id="head-5"></a>

The last step model saved in the `$USER_EXPERIMENT_DIR/isbi_experiment_unpruned/weights` dir is used for evaluation/ inference/ export. The evaluation also creates `$LOCAL_EXPERIMENT_DIR/isbi_experiment_unpruned/results_tlt.json`

In [ ]:
!tao unet evaluate --gpu_index=$GPU_INDEX -e $SPECS_DIR/spec_vanilla_unet_train.txt \
                 -m $USER_EXPERIMENT_DIR/semseg_experiment_unpruned/weights/freespace_unpruned.tlt\
                 -o $USER_EXPERIMENT_DIR/semseg_experiment_unpruned/ \
                 -k tlt_encode

In [ ]:
!cat $LOCAL_EXPERIMENT_DIR/semseg_experiment_unpruned/results_tlt.json

## 6. Visualizing Inferences <a class="anchor" id="head-6"></a>
In this section, we run the UNet inference tool to generate inferences on the trained models and print the results. 

The following cell will run inference for segmentation and visualize masks for the images in test. The resulting visualized images will be saved in the `vis_overlay_tlt` folder and label PNG masks in `mask_labels_tlt` in the path provided to `-o` argument.


In [ ]:
!tao unet inference --gpu_index=$GPU_INDEX -e $SPECS_DIR/spec_vanilla_unet_train.txt \
                 -m $USER_EXPERIMENT_DIR/semseg_experiment_unpruned/weights/freespace_unpruned.tlt \
                 -o $USER_EXPERIMENT_DIR/semseg_experiment_unpruned/ \
                 -k $KEY

In [ ]:
# Visualizing the sample images.
inference_folder = os.environ["LOCAL_EXPERIMENT_DIR"] + '/semseg_experiment_unpruned/vis_overlay_tlt' 

visualize_images(inference_folder)

## 7. Prune the trained model <a class="anchor" id="head-7"></a>
* Specify pre-trained model
* Equalization criterion
* Threshold for pruning.
* A key to save and load the model
* Output directory to store the model

*Usually, you just need to adjust `-pth` (threshold) for accuracy and model size trade off. Higher `pth` gives you smaller model (and thus higher inference speed) but worse accuracy. The threshold to use is dependent on the dataset. A pth value `5.2e-6` is just a start point. If the retrain accuracy is good, you can increase this value to get smaller models. Otherwise, lower this value to get better accuracy.*

* For some internal studies, we have noticed that a pth value of 0.1 is a good starting point for unet models trained on larger datasets. A larger regularization value in the first round of training will result in smaller models while pruning. Hence regularization while training and pth are hyper-parameters that needs to be tuned.*

In [ ]:
# Create an output directory if it doesn't exist.
!mkdir -p $LOCAL_EXPERIMENT_DIR/semseg_experiment_pruned

In [ ]:
!tao unet prune \
                  -e $SPECS_DIR/spec_vanilla_unet_train.txt \
                  -m $USER_EXPERIMENT_DIR/semseg_experiment_unpruned/weights/freespace_unpruned.tlt \
                  -o $USER_EXPERIMENT_DIR/semseg_experiment_pruned/model_freespace_pruned.tlt \
                  -eq union \
                  -pth 0.1 \
                  -k $KEY

In [ ]:
!ls -rlt $LOCAL_EXPERIMENT_DIR/semseg_experiment_pruned/

## 8. Retrain the pruned model <a class="anchor" id="head-8"></a>
* Model needs to be re-trained to bring back accuracy after pruning
* Specify re-training specification with pretrained weights as pruned model.

*Note: For retraining, please set the `load_graph` option to `true` in the model_config to load the pruned model graph. Also, if after retraining, the model shows some decrease in MIOU, it could be that the originally trained model was pruned a little too much. Please try reducing the pruning threshold (thereby reducing the pruning ratio) and use the new model to retrain.*

*Note: Ensure to provide a different folder for saving results of retraining from the folder where pruned model is saved.

In [ ]:
# Printing the retrain experiment file. 
# Note: We have updated the experiment file to include the 
# newly pruned model as a pretrained weights and, the
# load_graph option is set to true 
!cat $LOCAL_SPECS_DIR/spec_vanilla_unet_prune_retrain.txt

In [ ]:
# Retraining using the pruned model as pretrained weights 
!tao unet train --gpus=1 --gpu_index=$GPU_INDEX \
              -e $SPECS_DIR/spec_vanilla_unet_prune_retrain.txt \
              -r $USER_EXPERIMENT_DIR/semseg_experiment_retrain \
              -m $USER_EXPERIMENT_DIR/semseg_experiment_pruned/model_freespace_pruned.tlt \
              -n model_freespace_retrained \
              -k $KEY 

In [ ]:
# Listing the newly retrained model.
!ls -rlt $LOCAL_EXPERIMENT_DIR/semseg_experiment_retrain

## 9. Evaluate the retrained model <a class="anchor" id="head-9"></a>

In [ ]:
!tao unet evaluate --gpu_index=$GPU_INDEX -e $SPECS_DIR/spec_vanilla_unet_prune_retrain.txt \
                 -m $USER_EXPERIMENT_DIR/semseg_experiment_retrain/weights/model_freespace_retrained.tlt \
                 -o $USER_EXPERIMENT_DIR/semseg_experiment_retrain/ \
                 -k $KEY

In [ ]:
!tao unet inference --gpu_index=$GPU_INDEX -e $SPECS_DIR/spec_vanilla_unet_prune_retrain.txt \
                 -m $USER_EXPERIMENT_DIR/semseg_experiment_retrain/weights/model_freespace_retrained.tlt \
                 -o $USER_EXPERIMENT_DIR/semseg_experiment_retrain/validation_images \
                 -k $KEY

## 10. Fine-Tune on Real - World Data <a class="anchor" id="head-10"></a>
* For completing this step, you will need few labeled real world images
* The format should be same as the data provided earlier with TAO (Images and Labels in Image format)
* Place the folders in the Local Experiments Train Directory, dataset paths should be updated correspondingly in the spec file which will be used for training

In [ ]:
!tao unet train --gpus=1 --gpu_index=$GPU_INDEX \
              -e $SPECS_DIR/spec_vanilla_unet_rw_finetune.txt \
              -r $USER_EXPERIMENT_DIR/semseg_experiment_rw_finetune \
              -m $USER_EXPERIMENT_DIR/semseg_experiment_retrain/weights/model_freespace_retrained.tlt \
              -n model_freespace_rw_finetune \
              -k $KEY 

## 11. Evaluate the retrained model <a class="anchor" id="head-11"></a>

This section evaluates the pruned and retrained model, using the `evaluate` command.

In [ ]:
!tao unet evaluate --gpu_index=$GPU_INDEX -e $SPECS_DIR/spec_vanilla_unet_rw_finetune.txt \
                 -m $USER_EXPERIMENT_DIR/semseg_experiment_rw_finetune/weights/model_freespace_rw_finetune.tlt \
                 -o $USER_EXPERIMENT_DIR/semseg_experiment_rw_finetune/ \
                 -k $KEY

## 12. Model Export <a class="anchor" id="head-12"></a>

In [ ]:
# tao <task> export will fail if .etlt already exists. So we clear the export folder before tao <task> export
!rm -rf $LOCAL_EXPERIMENT_DIR/export
# # Export in FP32 mode. 
!mkdir -p $LOCAL_EXPERIMENT_DIR/export 

!tao unet export --gpu_index=$GPU_INDEX -m $USER_EXPERIMENT_DIR/semseg_experiment_rw_finetune/weights/model_freespace_rw_finetune.tlt \
                -o $USER_EXPERIMENT_DIR/export/model_freespace_rw_finetune.etlt \
                -k $KEY \
                -e $SPECS_DIR/spec_vanilla_unet_rw_finetune.txt \
                --data_type int8 \
                --batches 10 \
                --cal_image_dir $DATA_DOWNLOAD_DIR/real_world_images \
                --engine_file $USER_EXPERIMENT_DIR/export/model_freespace_rw_finetune_int8.engine \
                --cal_cache_file  $USER_EXPERIMENT_DIR/export/model_freespace_rw_finetune_int8_cal.bin \
                --cal_data_file $USER_EXPERIMENT_DIR/export/model_freespace_rw_finetune_int8_data.txt \
                --max_batch_size 4 \
                --batch_size 1

In [ ]:
# Check if etlt model is correctly saved.
!ls -l $LOCAL_EXPERIMENT_DIR/export/

Verify engine generation using the `tao-converter` utility included in the docker.

The `tao-converter` produces optimized tensorrt engines for the platform that it resides on. Therefore, to get maximum performance, please instantiate this docker and execute the `tao-converter` command, with the exported `.etlt` file and calibration cache (for int8 mode) on your target device. The tao-converter utility included in this docker only works for x86 devices, with discrete NVIDIA GPU's. 

For the jetson devices, please download the tao-converter for jetson from the dev zone link [here](https://developer.nvidia.com/tao-converter). 

If you choose to integrate your model into deepstream directly, please refer to [deepstream dev guide](https://docs.nvidia.com/metropolis/deepstream/dev-guide/index.html) for more details.

In [ ]:
!tao converter -k $KEY  \
               -e $USER_EXPERIMENT_DIR/export/model_freespace_rw_finetune_int8_trt.engine \
               -c $USER_EXPERIMENT_DIR/export/model_freespace_rw_finetune_int8_cal.bin \
               -t int8 \
               -p input_1,1x3x512x512,2x3x512x512,4x3x512x512 \
               -m $USER_EXPERIMENT_DIR/export/model_freespace_rw_finetune.etlt

In [ ]:
print('Exported engine:')
print('------------')
!ls -lh $LOCAL_EXPERIMENT_DIR/export/

## 13. Verify the deployed model <a class="anchor" id="head-13"></a>

Verify the converted engine. The resulting TRT inference images will be saved in `vis_overlay_trt` folder and PNG masks in `mask_labels_trt`  in the path provided to `-o` argument.

In [ ]:
!tao unet inference --gpu_index=$GPU_INDEX -e $SPECS_DIR/spec_vanilla_unet_rw_finetune.txt \
                 -m $USER_EXPERIMENT_DIR/export/model_freespace_rw_finetune_int8.engine \
                 -o $USER_EXPERIMENT_DIR/semseg_experiment_rw_finetune/ \
                 -k $KEY

In [ ]:
!tao unet evaluate --gpu_index=$GPU_INDEX -e $SPECS_DIR/spec_vanilla_unet_rw_finetune.txt \
                 -m $USER_EXPERIMENT_DIR/export/model_freespace_rw_finetune_int8.engine \
                 -o $USER_EXPERIMENT_DIR/semseg_experiment_rw_finetune/ \
                 -k $KEY

In [ ]:
# Visualizing the sample images.
inference_folder = os.environ["LOCAL_EXPERIMENT_DIR"] + '/semseg_experiment_re_finetune/vis_overlay_tlt' 

visualize_images(inference_folder)

Now lets deploy trained model with real world images on Jetson with Isaac ROS. Steps are included in README on GitHub.

![Isaac ROS Output](images/isaac_ros_output.png)
